In [222]:
%load_ext autoreload
%autoreload 2

import clean_data
import pandas as pd
import modeling_2 as model
import numpy as np
import matplotlib.pyplot as plt
import feature_selection as fs
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
merged_df = clean_data.runAll()
merged_df_no_id = merged_df.drop('UNITID', axis=1)

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [207]:
test_baseline = merged_df[['UNITID','MD_EARN_WNE_P6', 'Year']]
test_baseline = test_baseline[test_baseline['Year'].isin([2013, 2014])]
test_baseline.reset_index()
test_baseline.sort_values(by=['UNITID', 'Year'])
test_baseline = test_baseline.rename(columns={"MD_EARN_WNE_P6": "TEST_MD_EARN_WNE_P6"})

In [208]:
target = 'MD_EARN_WNE_P6'
merged_df_baseline_2013 = pd.DataFrame(merged_df[merged_df['Year'] == 2012]) 
merged_df_baseline_2013 = merged_df_baseline_2013[['MD_EARN_WNE_P6', 'Year', 'UNITID']]
merged_df_baseline_2013[target] = merged_df_baseline_2013[target].apply(lambda x: x + np.log(1.03))
merged_df_baseline_2013['Year'] = 2013
merged_df_baseline_2013.reset_index()

merged_df_baseline_2014 = merged_df_baseline_2013.copy()
merged_df_baseline_2014[target] = merged_df_baseline_2013[target].apply(lambda x: x + np.log(1.03))
merged_df_baseline_2014['Year'] = 2014
merged_df_baseline_2014.reset_index()

merged_baseline = pd.concat([merged_df_baseline_2013, merged_df_baseline_2014])
merged_baseline.sort_values(by=['UNITID', 'Year'])
merged_baseline=merged_baseline.rename(columns={"MD_EARN_WNE_P6": "PRED_MD_EARN_WNE_P6"})
merged_baseline.columns

Index(['PRED_MD_EARN_WNE_P6', 'Year', 'UNITID'], dtype='object')

In [210]:
complete = pd.merge(test_baseline, merged_baseline, how='left', on=['UNITID', 'Year'])

In [219]:
complete = complete.dropna()
complete.isna().sum()

UNITID                 0
TEST_MD_EARN_WNE_P6    0
Year                   0
PRED_MD_EARN_WNE_P6    0
dtype: int64

In [220]:
mean_squared_error(complete['TEST_MD_EARN_WNE_P6'], complete['PRED_MD_EARN_WNE_P6'])

0.004897730972596517

In [226]:
mean_absolute_error(complete['TEST_MD_EARN_WNE_P6'].apply(lambda x: np.exp(x)), \
                                complete['PRED_MD_EARN_WNE_P6'].apply(lambda x: np.exp(x)))

1313.7888732394367